# Tuning: Moving average inference approach

In [1]:
import shap
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, make_scorer


from glob import glob
import os
import optuna
import joblib
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

date = str(datetime.date(datetime.now()))
print(date)

2022-05-23


In [2]:
import psutil
    
ram_gb = psutil.virtual_memory().total / 2**30 # total physical memory in bytes
print(f"RAM: {ram_gb:.2f} GB")

RAM: 251.79 GB


In [3]:
# import
window = 60
csv_paths = glob(f'dataset/per-vehicle-moving-average/window-{window}/*.csv')

df = pd.concat([pd.read_csv(path) for path in csv_paths])

In [4]:
# quick eda on some cols
inspect_cols = ['accel', 'decel', 'speed_kph', 'elevation']
df[inspect_cols].describe()

,accel,decel,speed_kph,elevation
count,3.090632e+07,3.090632e+07,3.090632e+07,3.090632e+07
mean,1.272281e+00,1.446541e+00,4.175950e+01,5.860810e+01
std,1.630303e+00,1.749992e+00,1.885087e+01,5.345039e+00
min,0.000000e+00,0.000000e+00,2.000000e+01,3.600000e+01
25%,0.000000e+00,0.000000e+00,3.000000e+01,5.500000e+01
50%,1.000000e+00,1.000000e+00,3.000000e+01,5.900000e+01
75%,2.000000e+00,2.000000e+00,6.000000e+01,6.200000e+01
max,2.000000e+01,2.000000e+01,8.000000e+01,2.070000e+02


In [5]:
def clean_data(df):
    df['barangay'] = df['barangay'].fillna('Out-of-town')
    df = df.loc[df['barangay'] != 'Out-of-town']
    df['lanes'] = df['lanes'].fillna(1.0)
    return df

def filter_data(df, thresh):
    df = df.loc[df['num_periods'] >= thresh]
    return df

In [6]:
df = clean_data(df)

In [7]:
above_thresh_percentage =((df['num_periods'] >= 12).sum() / df.shape[0]) * 100
print("Percentage of samples with period over 12: ", above_thresh_percentage)

Percentage of samples with period over 12:  3.27421963137833


In [8]:
df = filter_data(df, thresh=12)

In [9]:
%%time
tuning_size = 0.30 # 30% of the training set is used for tuning

retrain, tuning_sample = train_test_split(df, test_size=tuning_size, random_state=11)
train, test = train_test_split(tuning_sample, test_size=0.1, random_state=11)

save_dir = f'dataset/per-vehicle-moving-average/inference/{date}'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

test.to_csv(f'{save_dir}/window-{window}-test.csv', index=False)
train.to_csv(f'{save_dir}/window-{window}-train.csv', index=False)
retrain.to_csv(f'{save_dir}/window-{window}-retrain.csv', index=False)

CPU times: user 13.1 s, sys: 211 ms, total: 13.3 s
Wall time: 13.7 s


In [10]:
print("Test shape: ", test.shape)
print("Train shape: ", train.shape)
print("Retrain shape: ", retrain.shape)

Test shape:  (30357, 27)
Train shape:  (273213, 27)
Retrain shape:  (708330, 27)


In [11]:
# use for ML
cat_cols = ['hour', 'dayofweek']
cat_cols_index = [i for i in range(len(cat_cols))]
num_cols = ['pix_business', 'pix_residential', 'pix_industrial', 
            'lanes', 'speed_kph', 'elevation']

In [12]:
# prepare data from train: previously used old vehicle speed, 
# use recomputed speed starting 2022-05-23
X, y = train[cat_cols + num_cols], train['recomputed_speed']

# prepare kfold
kf = KFold(n_splits=10)

# prepare rmse scorer
def rmse(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return rmse
rmse_scorer = make_scorer(rmse) # rmse scoring metric for cross_val

In [13]:
def objective(trial):
    lgb_params = {
        'boosting_type': 'goss',
        'objective':'tweedie',
        'metric' : 'rmse',
        'seed':11,
        'verbose': -1,
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'n_estimators': trial.suggest_int("n_estimators", 100, 1000),
        'learning_rate' : trial.suggest_float("learning_rate", 0.001, 1, log=True)
    }
    
    top_rate =  trial.suggest_uniform("top_rate", 0, 1.0)
    other_rate = trial.suggest_uniform("other_rate", 0, (1.0 - top_rate)) 
    
    # corrects the top_rate/other_rate values to only sum to 1
    if (top_rate + other_rate) > 1.0:
        lgb_params['other_rate'] = other_rate / (top_rate + other_rate)
        lgb_params['top_rate'] = top_rate / (top_rate + other_rate)
        

    model = lgb.LGBMRegressor(**lgb_params)
    scores = cross_val_score(estimator=model, X=X, y=y, scoring=rmse_scorer, cv=kf, n_jobs=-1, 
                             fit_params={'categorical_feature': cat_cols_index},
                             error_score='raise')
    return np.mean(scores)

In [15]:
study_name = f'moving-average-inference-window-{window}-{date}'

save_dir = f'per-vehicle-moving-average-experiments/inference/{date}'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
study = optuna.create_study(study_name=study_name,
                                direction='minimize',
                                storage=f'sqlite:///{save_dir}/window-{window}.db',
                                load_if_exists=True)

[I 2022-05-23 16:34:34,315] A new study created in RDB with name: moving-average-inference-window-60-2022-05-23


In [16]:
%%time
study.optimize(objective, n_trials=30)

/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using ca

/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using ca

/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using ca

/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using ca

/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using ca

/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using ca

/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using ca

/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mdorosan/.local/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using ca

CPU times: user 6.7 s, sys: 6.57 s, total: 13.3 s
Wall time: 34min 43s
